In [1]:
from __future__ import print_function, division

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class LSGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(z, valid)
        # (!!!) Optimize w.r.t. MSE loss instead of crossentropy
        self.combined.compile(loss='mse', optimizer=optimizer)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = fashion_mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, global_step,d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)



    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_lsgan_fashion_mnist'):
            os.mkdir('images_lsgan_fashion_mnist')
        fig.savefig("images_lsgan_fashion_mnist/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    gan = LSGAN()
    gan.train(epochs=40, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.314850, acc.: 39.06%] [G loss: 0.281866]
epoch:0 step:2 [D loss: 0.249928, acc.: 53.91%] [G loss: 0.328754]
epoch:0 step:3 [D loss: 0.238551, acc.: 53.12%] [G loss: 0.300702]
epoch:0 step:4 [D loss: 0.208830, acc.: 70.31%] [G loss: 0.298793]
epoch:0 step:5 [D loss: 0.207171, acc.: 67.19%] [G loss: 0.311713]
epoch:0 step:6 [D loss: 0.183434, acc.: 68.75%] [G loss: 0.336693]
epoch:0 step:7 [D loss: 0.172553, acc.: 75.00%] [G loss: 0.356387]
epoch:0 step:8 [D loss: 0.186004, acc.: 71.09%] [G loss: 0.360509]
epoch:0 step:9 [D loss: 0.167336, acc.: 81.25%] [G loss: 0.350154]
epoch:0 step:10 [D loss: 0.197334, acc.: 70.31%] [G loss: 0.365294]
epoch:0 step:11 [D loss: 0.182676, acc.: 75.00%] [G loss: 0.381655]
epoch:0 step:12 [D loss: 0.163693, acc.: 78.12%] [G loss: 0.419036]
epoch:0 step:13 [D loss: 0.164487, acc.: 75.78%] [G loss: 0.363244]
epoch:0 step:14 [D loss: 0.174652, acc.: 76.56%] [G loss: 0.404243]
epoch:0 step:15 [D loss: 0.173998, acc.: 71.09%] [G loss:

epoch:0 step:124 [D loss: 0.289338, acc.: 46.09%] [G loss: 0.393073]
epoch:0 step:125 [D loss: 0.250224, acc.: 61.72%] [G loss: 0.394401]
epoch:0 step:126 [D loss: 0.263195, acc.: 59.38%] [G loss: 0.328207]
epoch:0 step:127 [D loss: 0.252397, acc.: 57.81%] [G loss: 0.321680]
epoch:0 step:128 [D loss: 0.237877, acc.: 62.50%] [G loss: 0.344814]
epoch:0 step:129 [D loss: 0.293350, acc.: 50.00%] [G loss: 0.303204]
epoch:0 step:130 [D loss: 0.274623, acc.: 55.47%] [G loss: 0.428913]
epoch:0 step:131 [D loss: 0.233291, acc.: 60.94%] [G loss: 0.382323]
epoch:0 step:132 [D loss: 0.262333, acc.: 60.16%] [G loss: 0.428073]
epoch:0 step:133 [D loss: 0.273514, acc.: 57.03%] [G loss: 0.358978]
epoch:0 step:134 [D loss: 0.290392, acc.: 52.34%] [G loss: 0.381754]
epoch:0 step:135 [D loss: 0.277717, acc.: 58.59%] [G loss: 0.378144]
epoch:0 step:136 [D loss: 0.239116, acc.: 60.16%] [G loss: 0.409796]
epoch:0 step:137 [D loss: 0.250929, acc.: 59.38%] [G loss: 0.393274]
epoch:0 step:138 [D loss: 0.271794

...

epoch:39 step:36901 [D loss: 0.232966, acc.: 60.94%] [G loss: 0.286723]
epoch:39 step:36902 [D loss: 0.247844, acc.: 56.25%] [G loss: 0.308274]
epoch:39 step:36903 [D loss: 0.252167, acc.: 53.91%] [G loss: 0.328635]
epoch:39 step:36904 [D loss: 0.262947, acc.: 50.78%] [G loss: 0.279324]
epoch:39 step:36905 [D loss: 0.230655, acc.: 60.94%] [G loss: 0.303401]
epoch:39 step:36906 [D loss: 0.234089, acc.: 58.59%] [G loss: 0.279319]
epoch:39 step:36907 [D loss: 0.270114, acc.: 46.09%] [G loss: 0.256324]
epoch:39 step:36908 [D loss: 0.228832, acc.: 65.62%] [G loss: 0.287416]
epoch:39 step:36909 [D loss: 0.238044, acc.: 57.81%] [G loss: 0.283919]
epoch:39 step:36910 [D loss: 0.226213, acc.: 57.81%] [G loss: 0.310458]
epoch:39 step:36911 [D loss: 0.235712, acc.: 58.59%] [G loss: 0.317019]
epoch:39 step:36912 [D loss: 0.226504, acc.: 59.38%] [G loss: 0.292073]
epoch:39 step:36913 [D loss: 0.249072, acc.: 56.25%] [G loss: 0.313137]
epoch:39 step:36914 [D loss: 0.247724, acc.: 57.03%] [G loss: 0.

epoch:39 step:37018 [D loss: 0.216793, acc.: 68.75%] [G loss: 0.301259]
epoch:39 step:37019 [D loss: 0.232478, acc.: 60.16%] [G loss: 0.311886]
epoch:39 step:37020 [D loss: 0.238343, acc.: 57.81%] [G loss: 0.286879]
epoch:39 step:37021 [D loss: 0.244193, acc.: 60.16%] [G loss: 0.315087]
epoch:39 step:37022 [D loss: 0.239217, acc.: 57.03%] [G loss: 0.293187]
epoch:39 step:37023 [D loss: 0.227991, acc.: 65.62%] [G loss: 0.326859]
epoch:39 step:37024 [D loss: 0.237153, acc.: 60.16%] [G loss: 0.292318]
epoch:39 step:37025 [D loss: 0.245087, acc.: 56.25%] [G loss: 0.278605]
epoch:39 step:37026 [D loss: 0.267024, acc.: 49.22%] [G loss: 0.324827]
epoch:39 step:37027 [D loss: 0.246767, acc.: 53.12%] [G loss: 0.322403]
epoch:39 step:37028 [D loss: 0.242923, acc.: 52.34%] [G loss: 0.292234]
epoch:39 step:37029 [D loss: 0.212507, acc.: 70.31%] [G loss: 0.324244]
epoch:39 step:37030 [D loss: 0.250160, acc.: 56.25%] [G loss: 0.293429]
epoch:39 step:37031 [D loss: 0.243573, acc.: 57.03%] [G loss: 0.

epoch:39 step:37135 [D loss: 0.242357, acc.: 58.59%] [G loss: 0.312795]
epoch:39 step:37136 [D loss: 0.256999, acc.: 49.22%] [G loss: 0.301623]
epoch:39 step:37137 [D loss: 0.235874, acc.: 61.72%] [G loss: 0.309560]
epoch:39 step:37138 [D loss: 0.262673, acc.: 47.66%] [G loss: 0.294862]
epoch:39 step:37139 [D loss: 0.240840, acc.: 57.81%] [G loss: 0.270221]
epoch:39 step:37140 [D loss: 0.254728, acc.: 54.69%] [G loss: 0.302596]
epoch:39 step:37141 [D loss: 0.246777, acc.: 53.12%] [G loss: 0.301021]
epoch:39 step:37142 [D loss: 0.256664, acc.: 49.22%] [G loss: 0.300711]
epoch:39 step:37143 [D loss: 0.241063, acc.: 53.91%] [G loss: 0.300633]
epoch:39 step:37144 [D loss: 0.242585, acc.: 59.38%] [G loss: 0.292376]
epoch:39 step:37145 [D loss: 0.242022, acc.: 59.38%] [G loss: 0.280909]
epoch:39 step:37146 [D loss: 0.234591, acc.: 60.94%] [G loss: 0.291926]
epoch:39 step:37147 [D loss: 0.235799, acc.: 56.25%] [G loss: 0.302257]
epoch:39 step:37148 [D loss: 0.248406, acc.: 52.34%] [G loss: 0.

epoch:39 step:37250 [D loss: 0.235529, acc.: 60.16%] [G loss: 0.310245]
epoch:39 step:37251 [D loss: 0.263144, acc.: 46.09%] [G loss: 0.279170]
epoch:39 step:37252 [D loss: 0.278815, acc.: 42.97%] [G loss: 0.278677]
epoch:39 step:37253 [D loss: 0.238932, acc.: 58.59%] [G loss: 0.331993]
epoch:39 step:37254 [D loss: 0.238271, acc.: 64.06%] [G loss: 0.296673]
epoch:39 step:37255 [D loss: 0.227463, acc.: 57.81%] [G loss: 0.336926]
epoch:39 step:37256 [D loss: 0.240341, acc.: 58.59%] [G loss: 0.298338]
epoch:39 step:37257 [D loss: 0.240885, acc.: 57.81%] [G loss: 0.280013]
epoch:39 step:37258 [D loss: 0.250582, acc.: 57.81%] [G loss: 0.296980]
epoch:39 step:37259 [D loss: 0.238034, acc.: 62.50%] [G loss: 0.297243]
epoch:39 step:37260 [D loss: 0.236230, acc.: 57.81%] [G loss: 0.294221]
epoch:39 step:37261 [D loss: 0.226189, acc.: 63.28%] [G loss: 0.299840]
epoch:39 step:37262 [D loss: 0.249712, acc.: 52.34%] [G loss: 0.267438]
epoch:39 step:37263 [D loss: 0.248611, acc.: 60.16%] [G loss: 0.

epoch:39 step:37367 [D loss: 0.236147, acc.: 63.28%] [G loss: 0.265060]
epoch:39 step:37368 [D loss: 0.238891, acc.: 62.50%] [G loss: 0.286946]
epoch:39 step:37369 [D loss: 0.235617, acc.: 57.81%] [G loss: 0.269940]
epoch:39 step:37370 [D loss: 0.221494, acc.: 66.41%] [G loss: 0.298223]
epoch:39 step:37371 [D loss: 0.247174, acc.: 50.00%] [G loss: 0.309491]
epoch:39 step:37372 [D loss: 0.243384, acc.: 60.16%] [G loss: 0.278943]
epoch:39 step:37373 [D loss: 0.240911, acc.: 61.72%] [G loss: 0.286737]
epoch:39 step:37374 [D loss: 0.256259, acc.: 43.75%] [G loss: 0.277729]
epoch:39 step:37375 [D loss: 0.258666, acc.: 48.44%] [G loss: 0.312664]
epoch:39 step:37376 [D loss: 0.257817, acc.: 49.22%] [G loss: 0.308015]
epoch:39 step:37377 [D loss: 0.244178, acc.: 57.81%] [G loss: 0.329987]
epoch:39 step:37378 [D loss: 0.233697, acc.: 65.62%] [G loss: 0.322862]
epoch:39 step:37379 [D loss: 0.241943, acc.: 55.47%] [G loss: 0.284953]
epoch:39 step:37380 [D loss: 0.248061, acc.: 53.12%] [G loss: 0.